## Some imports

In [1]:
from PIL import Image
import numpy as np

from ipywidgets import (interact)

import bokeh
from bokeh.plotting import (figure, show, output_notebook)
from bokeh.io import (push_notebook)

output_notebook()

Loading BokehJS ...

## Load the data

![The Astronomer -- by Vermeer](./vermeer.jpg)

In [2]:
img = Image.open('./vermeer.jpg').convert('L')
fdata = np.array(img.getdata()).astype('float32').reshape(img.size[::-1])
fdata = (fdata - fdata.min()) / (fdata.max() - fdata.min())
fdata.shape

# invert y-axis
data = fdata[::-1,:]

In [3]:
p = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=640, plot_height=700)
p.image(image=[data], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.viridis(255))
show(p)

In [4]:
from hyper_canny import (smooth_sobel, edge_thinning, double_threshold, cp_edge_thinning, cp_double_threshold)

### Sobel filter

In [5]:
sobel = smooth_sobel(data.copy(), 5, 2.0)

In [6]:
sobel_data = 1.0/sobel[:,:,2]

fig_sobel = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=640, plot_height=700)
img = fig_sobel.image(image=[sobel_data], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette='Viridis256')
show(fig_sobel)

### Edge thinning

In [7]:
thin_edges = cp_edge_thinning(sobel)

In [8]:
fig_edges = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=640, plot_height=700)
img = fig_edges.image(image=[thin_edges], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.grey(2)[::-1])
show(fig_edges)

### Hysteresis thresholding

In [9]:
hysteric = cp_double_threshold(sobel, thin_edges, 150., 200.)

In [10]:
fig_hyst = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=640, plot_height=700)
img = fig_hyst.image(image=[hysteric], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.grey(2)[::-1])
show(fig_hyst)

## Putting it together

In [13]:
fig = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=500, plot_height=600)
img = fig.image(image=[data], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.grey(3)[::-1])

s = None
m = None
s_state = 0.0

def update(sigma, low, high):
    global s, m, s_state
    if abs(sigma - s_state) > 0.1:
        s_state = sigma
        # print("sobel", flush=True)
        s = smooth_sobel(data.copy(), int(np.floor(sigma*2)), sigma)
        # print("thinning edges", flush=True)
        m = edge_thinning(s)
    # print("double threshold", flush=True)
    c = cp_double_threshold(s, m, low, high)
    img.data_source.data['image'] = [c]
    push_notebook()

In [14]:
show(fig, notebook_handle=True)
interact(update, sigma=(0., 10.), low=(0., 200.), high=(0., 200.))

<function __main__.update>